In [42]:
import json
from functools import lru_cache
from typing import List

import pandas as pd
import requests
import seaborn as sns
# from haversine import haversine
from pydantic import BaseModel, confloat, AnyUrl
from ratelimit import RateLimitException, limits
from tenacity import *
from yarl import URL

In [43]:
API_ENDPOINT = URL("https://api.coingecko.com/api/v3")

In [44]:
@retry(
    retry=retry_if_exception_type(requests.exceptions.RequestException),
    wait=wait_exponential(max=60),
    stop=stop_after_attempt(3),
)
@retry(
    retry=retry_if_exception_type(RateLimitException),
    wait=wait_fixed(60),
    stop=stop_after_delay(360),
)
@limits(calls=50, period=60)
def call_get(url: URL, params: dict) -> str:
    return requests.get(url.update_query(params)).text

In [38]:
dict_market_data = {'vs_currency' : 'usd', 'per_page' : '250'}

In [45]:
class MarketDataInfo(BaseModel):
    id: str
    symbol: str
    current_price: float
    market_cap: int
    image: AnyUrl



In [47]:
def parse_result(result) -> MarketDataInfo:
    return MarketDataInfo(
        id=result["id"],
        symbol=result["symbol"],
        current_price=result["current_price"],
        market_cap=result["market_cap"],
        image=result["image"]
    )

In [48]:
def get_market_data(page: int) -> List[str]:
    return [
        parse_result(result)
        for result in json.loads(
            call_get(API_ENDPOINT / "coins/markets", dict_market_data)
        )
    ]

In [50]:
def create_df() -> pd.DataFrame:
    return pd.DataFrame([item.dict() for item in get_market_data()])
